# Introduction to Graph Analysis with Spark GraphFrames
## Flight Graph / Network Analysis
Source Data: 
* [OpenFlights: Airport, airline and route data](http://openflights.org/data.html)
* [United States Department of Transportation: Bureau of Transportation Statistics (TranStats)](http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time)

Note the data used here was extracted from data between 1/1/2014 and 1/31/2014

In [ ]:
#get grameframes; it is not an official part of spark yet
#you only need to run this once with docker if you don't have it, and no need to do it on cluster
#!wget http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[4]') \
                    .config("spark.executor.memory", "1g") \
                    .config("spark.driver.memory", "1g") \
                    .config("spark.jars", "graphframes-0.8.1-spark3.0-s_2.12.jar") \
                    .config("spark.packages", "graphframes:graphframes:0.8.1-spark3.0-s_2.12") \
                    .appName('spark_graphframe').getOrCreate()

In [ ]:
#we need to add the jar file to use the package
sc = spark.sparkContext
sc.getConf().getAll()
sc.addPyFile('graphframes-0.8.1-spark3.0-s_2.12.jar')
#on cluster you can get it from (only available for second cluster)
#sc.addPyFile('s3://msbx5420-spr21/zhiyiwang/graphframes-0.8.1-spark3.0-s_2.12.jar')

### Data Preparation

In [ ]:
#load airports dataset
airports_na = spark.read.csv('./flight-data/airport-codes-na.txt', header='true', inferSchema='true', sep='\t')
airports_na.show()
airports_na.createOrReplaceTempView("airports_na")

#load flights departure delay data
departure_delays = spark.read.csv('./flight-data/departuredelays.csv', header='true', inferSchema='true')
departure_delays.show()
departure_delays.createOrReplaceTempView("departure_delays")
#departure_delays.cache()

In [ ]:
#use available IATA codes from the departuredelays to sample dataset
trip_IATA = spark.sql("select distinct IATA from (select distinct origin as iata from departure_delays union all select distinct destination as iata from departure_delays)")
trip_IATA.show()
trip_IATA.createOrReplaceTempView("trip_IATA")

#only include airports with at least one trip from the departure delays dataset
airports = spark.sql("select a.IATA, a.City, a.State, a.Country from airports_na a, trip_IATA b where a.IATA = b.IATA")
airports.show()
airports.createOrReplaceTempView("airports")
#airports.cache()

In [ ]:
departure_delays.show()
departure_delays.count()

In [ ]:
#build `departure_gelays_geo` dataframe
#obtain key attributes such as date of flight, delays, distance, and airport information (Origin, Destination)  
departure_delays_geo = spark.sql("select cast(a.date as int) as trip_id, \
                                 cast(concat('2014-', substr(cast(a.date as string), 1, 1), '-', substr(cast(a.date as string), 2, 2), ' ', \
                                 substr(cast(a.date as string), 4, 2), ':', substr(cast(a.date as string), 6, 2), ':00') as timestamp) as local_date, \
                                 cast(a.delay as int), cast(a.distance as int), a.origin as src, a.destination as dst, b.city as city_src, \
                                 c.city as city_dst, b.state as state_src, c.state as state_dst \
                                 from departure_delays a, airports b, airports c where a.origin = b.iata and a.destination = c.iata") 

departure_delays_geo.createOrReplaceTempView("departure_delays_geo")
departure_delays_geo.show()
#departure_delays_geo.cache()
departure_delays_geo.count()

### Building the Graph
Now that we've imported our data, we can build our graph. To do so we're going to do two things: we are going to build the structure of the vertices (or nodes) and the structure of the edges. The match the naming requirement, we will do:

* Rename IATA airport code to **id** in the vertices dadaframe
* Make sure start and end airports as **src** and **dst** for the edges dataframe (flights)

These are naming conventions for vertices and edges in GraphFrames.

In [ ]:
#make sure you have already added the GraphFrames package
import pyspark.sql.functions as fn
from graphframes import *

#create vertices (airports) and edges (flights)
trip_vertices = airports.withColumnRenamed("IATA", "id").distinct()
trip_edges = departure_delays_geo.select("trip_id", "delay", "src", "dst", "city_src", "city_dst", "state_src", "state_dst")

#trip_vertices.cache()
#trip_edges.cache()

In [ ]:
#build `trip_graph` GraphFrame
#this GraphFrame builds up on the vertices and edges based on our trips (flights)
trip_graph = GraphFrame(trip_vertices, trip_edges)
print(trip_graph)

#build `trip_graph_prime` GraphFrame
#this graphframe contains a smaller subset of data to make it easier to display motifs and subgraphs (later)
trip_edges_prime = departure_delays_geo.select("trip_id", "delay", "src", "dst")
trip_graph_prime = GraphFrame(trip_vertices, trip_edges_prime)

### Perform Queries on the Graph
Let's start with a set of simple graph queries to understand flight performance and departure delays

#### Determine the number of airports and trips

In [ ]:
print("Airports: {0}".format(trip_graph.vertices.count()))
print("Trips: {0}".format(trip_graph.edges.count()))

#### Determining the longest delay in this dataset

In [ ]:
trip_graph.edges.groupBy().max("delay").show()

#### Determining the number of delayed vs. on-time / early flights

In [ ]:
# Determining number of on-time / early flights vs. delayed flights
print("On-time or Early Flights: {0}".format(trip_graph.edges.filter("delay <= 0").count()))
print("Delayed Flights: {0}".format(trip_graph.edges.filter("delay > 0").count()))

#### What flights departing SEA are most likely to have significant delays
Note, delay can be <= 0 meaning the flight left on time or early

In [ ]:
trip_graph.edges.filter("src = 'SEA' and delay > 0").groupBy("src", "dst").avg("delay").sort(fn.desc("avg(delay)")).show(5)

#### Most popular flights

In [ ]:
#determine the most popular flights (single city hops)
top_trips = trip_graph.edges.groupBy("src", "dst").agg(fn.count("delay").alias("trips")) 
top_trips.orderBy(top_trips.trips.desc()).show()

### Vertex Degrees
* `inDegrees`: Incoming connections to the airport
* `outDegrees`: Outgoing connections from the airport 
* `degrees`: Total connections to and from the airport

Reviewing the various properties of the property graph to understand the incoming and outgoing connections between airports.

In [ ]:
#degrees
#the number of degrees - the number of incoming and outgoing connections - for various airports within this sample dataset
trip_graph.degrees.sort(fn.desc("degree")).show()

In [ ]:
#inDegrees
#tThe number of degrees - the number of incoming connections - for various airports within this sample dataset
trip_graph.inDegrees.sort(fn.desc("inDegree")).show()

In [ ]:
#outDegrees
#the number of degrees - the number of outgoing connections - for various airports within this sample dataset
trip_graph.outDegrees.sort(fn.desc("outDegree")).show(20)

### City / Flight Relationships through Motif Finding
To understand the complex relationship of city airports and their flights with each other, we can use motifs to find patterns of airports (i.e. vertices) connected by flights (i.e. edges). The result is a DataFrame in which the column names are given by the motif keys.

### What delays might we blame on SFO

In [ ]:
#we use trip_graph_prime to more easily display 
#with motif finding
#we show the associated edge (ab, bc) relationships 
#with the different the city / airports (a, b, c) where SFO is the connecting city (b)
#by ensuring that flight ab (i.e. the flight to SFO) occured before flight bc (i.e. flight leaving SFO)
#note that trip_id was generated based on time in the format of MMDDHHMM converted to int
#therefore, bc.tripid < ab.tripid + 10000 means the second flight (bc) occured within approx a day of the first flight (ab)
#but in reality, we should be more careful to link trips ab and bc.
motifs = trip_graph_prime.find("(a)-[ab]->(b); (b)-[bc]->(c)")\
                         .filter("(b.id = 'SFO') and (ab.delay > 300 or bc.delay > 300) and bc.trip_id > ab.trip_id and bc.trip_id < ab.trip_id + 10000")
motifs.show(truncate=False)

### Determining Airport Ranking / Importance using PageRank
There are a large number of flights and connections through these various airports included in this departure delay dataset.  With the `pageRank` algorithm, Spark iteratively traverses the graph and determines a rough estimate of how important the airport is.

In [ ]:
#determining airport ranking of importance using pageRank
pageranks = trip_graph.pageRank(resetProbability=0.15, maxIter=5)
pageranks.vertices.orderBy(pageranks.vertices.pagerank.desc()).show()

### Top Transfer Cities
Many airports are used as transfer points instead of the final Destination.  An easy way to calculate this is by calculating the ratio of inDegree (the number of flights to the airport) / outDegree (the number of flights leaving the airport).  Values close to 1 may indicate many transfers, whereas values < 1 indicate many outgoing flights and > 1 indicate many incoming flights.  Note, this is a simple calculation that does not take into account of timing or scheduling of flights, just the overall aggregate number within the dataset.

In [ ]:
#calculate the inDegrees (flights into the airport) and outDegrees (flights leaving the airport)
in_degree = trip_graph.inDegrees
out_degree = trip_graph.outDegrees

#calculate the degree_ratio (inDegrees/outDegrees)
degree_ratio = in_degree.join(out_degree, 'id').selectExpr("id", "double(inDegree)/double(outDegree) as degree_ratio")

#join back to the airports dataFrame
non_transfer_airports = degree_ratio.join(airports, degree_ratio.id == airports.IATA) \
                                    .selectExpr("id", "city", "degree_ratio") \
                                    .filter("degree_ratio < 0.9 or degree_ratio > 1.1")

#list out the city airports which have abnormal degree ratios.
non_transfer_airports.show()

In [ ]:
#join back to the airports dataframe
transfer_airports = degree_ratio.join(airports, degree_ratio.id == airports.IATA) \
                                .selectExpr("id", "city", "degree_ratio") \
                                .filter("degree_ratio >= 0.9 and degree_ratio <= 1.1")

#list out the top 10 transfer city airports
transfer_airports.orderBy("degree_ratio", ascending=False).show(10)

### Breadth First Search 
Breadth-first search (BFS) is designed to traverse the graph to quickly find the desired vertices (i.e. airports) and edges (i.e flights).  Let's try to find the shortest number of connections between cities based on the dataset.  Note, these examples do not take into account of time or distance, just rough estimates.

In [ ]:
#case 1: direct flights from Seattle to San Francisco 
filtered_paths = trip_graph.bfs(fromExpr = "id = 'SEA'", toExpr = "id = 'SFO'", maxPathLength = 1)
filtered_paths.show(truncate=False)

As you can see, there are a number of direct flights between Seattle and San Francisco.

In [ ]:
#case 2: direct flight from San Francisco and Buffalo
filtered_paths = trip_graph.bfs(fromExpr = "id = 'SFO'", toExpr = "id = 'BUF'", maxPathLength = 1)
filtered_paths.show()

But there are no direct flights between San Francisco and Buffalo.

In [ ]:
#case 3: one stop flights from San Francisco to Buffalo
filtered_paths = trip_graph.bfs(fromExpr = "id = 'SFO'", toExpr = "id = 'BUF'", maxPathLength = 2)
filtered_paths.show(truncate=False)

But there are flights from San Francisco to Buffalo with Minneapolis as the transfer point. So what are the most popular layovers between `SFO` and `BUF`?

In [ ]:
#display most popular layover cities by descending count
filtered_paths.groupBy("v1.id", "v1.City").count().orderBy(fn.desc("count")).show()

### Shortest Paths
We can get shortest Paths from all vertices to the vertices (landmarks) we specify.

In [ ]:
trip_graph.shortestPaths(landmarks=["DEN", "ATL"]).show()
trip_graph.shortestPaths(landmarks=["DEN", "BUF"]).show()

### Triangle Structure
Triangle structure is a useful concept in network structure.

In [ ]:
trip_graph.triangleCount().show()

### Graph Clustering
Label Propagation is the approach to cluster gprah / network.

In [ ]:
clusters = trip_graph.labelPropagation(maxIter=5)
clusters.orderBy(fn.asc('label')).show()

In [ ]:
clusters.select('label').distinct().count()

In [ ]:
clusters.select('label').groupBy('label').count().show()

In [ ]:
#components = trip_graph.stronglyConnectedComponents(maxIter=10)
#components.show()
#components.select('component').distinct().count()

## Visualize Network with Our Graph Analysis

In [ ]:
import networkx as nx
import pandas as pd

#turn the large network into a smaller one and create network from pandas
vertice = trip_graph.vertices.toPandas()
edges = trip_graph.edges.groupBy('src','dst').agg(fn.count('*').alias('flights'), fn.avg('delay').alias('avg_delay')).toPandas()
ranks = pageranks.vertices.toPandas()
labels = clusters.toPandas()
#connected = components.toPandas()

vertice.index = vertice['id']
ranks.index = ranks['id']
labels.index = labels['id']
#connected.index = connected['id']

ranks['pagerank'] = ranks['pagerank'] * 100
edges['flights'] = edges['flights'] / 100

graph = nx.from_pandas_edgelist(edges, 'src',  'dst', ['flights', 'avg_delay'])
nx.set_node_attributes(graph, pd.Series(vertice.id, index=vertice.id).to_dict(), 'label')
nx.set_node_attributes(graph, pd.Series(ranks.pagerank, index=ranks.id).to_dict(), 'size')
nx.set_node_attributes(graph, pd.Series(labels.label, index=labels.id).to_dict(), 'group')
#nx.set_node_attributes(graph, pd.Series(connected.component, index=connected.id).to_dict(), 'component')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = [20, 20]

nx.draw(graph)
plt.figure()
plt.show()

In [ ]:
#add node size by pagerank and edge width by flights
pos = nx.spring_layout(graph, iterations=100)
nx.draw_networkx_nodes(graph, pos = pos, node_size = list(nx.get_node_attributes(graph, 'size').values()))
nx.draw_networkx_edges(graph, pos = pos, width = list(nx.get_edge_attributes(graph, 'flights').values()))
nx.draw_networkx_labels(graph, pos = pos, font_color = 'r')
plt.figure()
plt.show()

In [ ]:
#add color for different clusters
pos = nx.spring_layout(graph, iterations=100)
nx.draw_networkx_nodes(graph, pos = pos, node_size = list(nx.get_node_attributes(graph, 'size').values()), node_color = list(nx.get_node_attributes(graph, 'group').values()), cmap = plt.cm.get_cmap('rainbow'))
nx.draw_networkx_edges(graph, pos = pos, width = list(nx.get_edge_attributes(graph, 'flights').values()))
nx.draw_networkx_labels(graph, pos = pos, font_color = 'r')
plt.show()

In [ ]:
!pip install pyvis

In [ ]:
#alternatively use pyvis to visualize interactive graph
from pyvis.network import Network

net = Network(height=800, width=800, notebook=True)
net.barnes_hut()
net.from_nx(graph, node_size_transf=(lambda x: x/10))
net.show('graph.html')